# Taking the derivative of a form in fenics

In this example we use the following form:
$$a(m,u,v) := \int_\Omega e^m \nabla u \cdot \nabla v dx.$$
We will show three methods for taking the derivative of $a$ with respect to $m$ in the direction of a function $h$. That is, 
$$\frac{da}{dm}h = \lim_{s \rightarrow 0}\frac{a(m+sh,u,v) - a(m,u,v)}{s}.$$
For notational convenience, we do not explicitly write the dependence of $\frac{da}{dm}h$ on $m,u,v$.

In [1]:
import numpy as np
from fenics import *

In [2]:
mesh = UnitSquareMesh(10,11)

V = FunctionSpace(mesh, 'CG', 1)

u = TrialFunction(V)
v = TestFunction(V)
m = Function(V)

a = inner(exp(m) * grad(u), grad(v))*dx

h = Function(V)
h.vector()[:] = np.random.randn(V.dim())

## Method 1: finite differences
We may approximate the derivative with the finite difference:
$$\frac{da}{dm}h \approx \frac{a(m+sh,u,v) - a(m,u,v)}{s}$$
for some small but finite step size $s$.

In [3]:
A1 = assemble(a).array() 

s = 1e-6 # finite difference step size
m.vector()[:] = m.vector() + s * h.vector()

A2 = assemble(a).array()

dA_finitediff = (A2 - A1)/s # Assembled derivative via finite differences

m.vector()[:] = m.vector() - s * h.vector() # Reset m to it's original state

Note: The command .array() in assemble(a).array() converts the sparse petsc matrix generated by the command assemble(a) into a dense numpy array.
We do this for convenience here since numpy arrays are easier to work with than petsc matrices. But converting a sparse matrix to a dense matrix is highly computationally inefficient, so you should use the petsc matrices in real problems.

## Method 2: analytical formula
From calculus, one can show that
$$\begin{aligned}
\frac{da}{dm}h &= \frac{d}{dm}\left(\int_\Omega e^m \nabla u \cdot \nabla v dx\right)h \\
&= \int_\Omega h e^m \nabla u \cdot \nabla v dx.
\end{aligned}$$

In [4]:
da_dm_h_analytic = inner(h * exp(m) * grad(u), grad(v))*dx

dA_analytic = assemble(da_dm_h_analytic).array() # Assembled derivative via analytical formula

error_analytic_vs_finitediff = np.linalg.norm(dA_finitediff - dA_analytic)/np.linalg.norm(dA_finitediff)
print('step size=', s, ', error_analytic_vs_finitediff=', error_analytic_vs_finitediff)

step size= 1e-06 , error_analytic_vs_finitediff= 7.339890987128867e-07


## Method 3: Automatic differentiation using fenics.derivative()

In [5]:
da_dm_h_autodiff = derivative(a, m, h)

dA_autodiff = assemble(da_dm_h_autodiff).array() # Assembled derivative via automatic differentiation

error_autodiff_vs_finitediff = np.linalg.norm(dA_finitediff - dA_autodiff)/np.linalg.norm(dA_finitediff)
print('step size=', s, ', error_autodiff_vs_finitediff=', error_autodiff_vs_finitediff)

step size= 1e-06 , error_autodiff_vs_finitediff= 7.339890987128867e-07


## You should use Method 3 (fenics.derivative()) whenever possible
Reasons:
### Accuracy 
 - Automatic differentiation is exact (to numerical precision). 
 - Analytical formulas may not be exact, because differentiation and discretization do not always commute. However, note that differentiation and discretization do commute for Galerkin discretizations, such as the one used here, so the analytic formula is exact in this example. 
 - Finite differences are not exact, owing to the finite step size. Choosing a good step size can be difficult.

### Computational efficiency
 - Automatic differentiation procedure will typically result in more computationally efficient code than implementation of an analytic formula that you derived by-hand. People have dedicated their careers, gotten PhD's, written books, etc, on the subject of optimizing the computational graph in automatic differentiation. 
 - Assembly of the derivative form generated by automatic differentiation is typically faster and uses less memory than assembly of the original form at two different locations, as is done in finite differences.
 
### Human efficiency
 - To use the analytical formula approach, you must spend a lot of time and effort doing calculus on pencil and paper, typing formulas, and checking your work/debugging. In contrast, for automatic differentiation you just call one function, and you know that it is going to be correct.

## Replace function

In [6]:
from ufl import replace

u = TrialFunction(V)
v = TestFunction(V)
u_fct = Function(V)
a = inner(grad(u), grad(v))*dx

In [7]:
print('u=', u)
print('v=', v)
print('u_fct=', u_fct)

u= v_1
v= v_0
u_fct= f_27


In [8]:
print('Functions in a')
for coeff in a.coefficients():
    print(coeff)
    
print('TestFunctions and TrialFunctionst in a=')
for arg in a.arguments():
    print(arg)

Functions in a
TestFunctions and TrialFunctionst in a=
v_0
v_1


In [9]:
a2 = replace(a, {u:u_fct})


In [10]:
print('Functions in a2')
for coeff in a2.coefficients():
    print(coeff)
    
print('TestFunctions and TrialFunctionst in a2=')
for arg in a2.arguments():
    print(arg)

Functions in a2
f_27
TestFunctions and TrialFunctionst in a2=
v_0
